In [13]:
!pip install -q geopandas
!pip install pydeck
!pip3 install --upgrade setuptools
!pip3 install shapely
!pip install xmltodict 

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\hwang in beom\Anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\pip-install-1qad4u4r\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\pip-install-1qad4u4r\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-pip-egg-info-603ju5jl'
         cwd: C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-install-1qad4u4r\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: pyth

In [151]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
# import geopandas as gpd # 공간정보 라이브러리
import pandas as pd
import os

import requests
import json


# 사용하는 디렉토리 정의
BASE_WORKING_DIR = os.getcwd()
DATA_RAW_PATH = os.path.join(BASE_WORKING_DIR)

In [28]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

## 도서관 행정동 데이터 생성

In [29]:
# 행정동 데이터 
goyang_geo_gdf = gpd.read_file(os.path.join(DATA_RAW_PATH, 'HangJeongDong_ver20210401.geojson'), encoding='cp949')

In [32]:
# 행정동 데이터 중 서울 데이터만 추출
seoul_geo_df = goyang_geo_gdf[goyang_geo_gdf['sidonm']=='서울특별시']

In [3]:
# 도서관 정보 데이터 
libry_info = pd.read_csv("C:/Users/hwang in beom/Desktop/도서관/도서관1/공공도서관_정보.csv")

In [12]:
# 도서관 데이터 중 서울 데이터만 추출
seoul_libry = libry_info[ (libry_info['address1']=='서울특별시') & (libry_info['lib_type_nm']=='공공') ] 

In [134]:
# 도서관 위/경도를 통해 해당하는 동 추출
dong_name = []
number = []
for i in range(0,len(seoul_libry)):
    for j in range(0,len(seoul_geo_df)):
        test_code1 = Point(seoul_libry.iloc[i]['lib_longitude'], seoul_libry.iloc[i]['lib_latitude'])
        if test_code1.within(seoul_geo_df.iloc[j]['geometry']):
            dong_name.append(seoul_geo_df.iloc[j]['adm_nm'])
#             print(i)
            number.append(i)
#             print(seoul_geo_df.iloc[j]['adm_nm'])
            break
    else:
        dong_name.append('서울특별시 서초구 신원동')

# 동 이름만 추출 + 도서관 데이터에 동 이름 추가 
dong=[]
for i in dong_name:
    dong.append(i.split()[-1])
seoul_libry['행정동'] = dong


In [148]:
seoul_libry.to_csv('seoul_libry.csv',encoding='utf-8-sig')

## 도서관 대출 데이터 

In [203]:
import xmltodict
import json

In [149]:
auth_key = '7d39c6d3dcf678df9fbe84c27a7d978e0e534a12568b2d8be32af2a2cd38e5b5'

In [194]:
#도서관 코드 
lib_code_url = 'http://data4library.kr/api/libSrch?authKey=7d39c6d3dcf678df9fbe84c27a7d978e0e534a12568b2d8be32af2a2cd38e5b5&pageNo=1&pageSize=10000'


In [195]:
res = requests.get(lib_code_url)
str_xml = res.text

In [200]:
xml_parse = xmltodict.parse(str_xml) # return collections.OrderedDict
json_parse = json.loads(json.dumps(cc)) # return dict

In [218]:
lib_list=[]
for i in range(0,len(json_parse['response']['libs']['lib'])):
    lib_list.append(list(json_parse['response']['libs']['lib'][i].values()))

In [222]:
column= list(json_parse['response']['libs']['lib'][i].keys())

In [224]:
lib_df = pd.DataFrame(lib_list,columns=column)

,libCode,libName,address,tel,fax,latitude,longitude,homepage,closed,operatingTime,BookCount
0,127058,2.28도서관,대구광역시 중구 2·28길 9,053-257-2280,053-257-2284,35.8592504,128.5894055,http://library.daegu.go.kr/228lib/index.do,"매주 월요일 / 법정공휴일(일요일을 제외한 관공서 공휴일), 도서관 및 기타 사정으...","화~금 09:00~18:00, 토일 09:00~17:00",51189
1,148096,U보라작은도서관,"경상남도 김해시 전하로176번길 71, 반도보라아파트 주민공동시설 2층",055-324-3335,-,35.223558,128.874226,http://lib.gimhae.go.kr/01293/01298/00212.web,"매주 일요일 / 법정공휴일, 임시공휴일",-,7229
2,111456,가락몰도서관,서울특별시 송파구 양재대로 932,02-3435-0950,02-3435-0959,37.492994,127.112326,http://www.splib.or.kr/spalib/,"매주 월요일 / 법정공휴일, 기타 도서관 사정에 의해 관장이 정한 날",09:00 ~ 18:00,32613
3,125010,가수원도서관,대전광역시 서구 가수원로 91-11,042-288-4770,042-288-5937,36.3019,127.348812,http://www.seogu.go.kr/learning/gasuwonlib/ind...,"매주 월요일 / 국경일, 정부에서 정한 공휴일, 도서관 사정으로 인한 휴관하는 날","(하계) 07:00~22:00, (동계) 08:00~22:00",156677
4,111287,가슴따뜻한작은도서관,서울특별시 서대문구 응암로1길 10,02-330-8652,-,37.5814481,126.9111617,http://lib.sdm.or.kr,"매주 토요일, 일요일 / 법정공휴일",월~금 09:00~18:00,6444
5,125012,가양도서관,대전광역시 동구 우암로277번길 72,042-259-7081,042-628-8168,36.3489771,127.4451507,https://www.donggu.go.kr/dg/lib,"매주 월요일 / 국경일, 기타 원장이 필요하다고 인정되는 경우","평일 08:00 ~ 22:00, 주말 08:00 ~ 18:00",142498
6,125008,가오도서관,대전광역시 동구 동구청로 147,042-259-7051,042-224-6171,36.3124314,127.4553237,https://www.donggu.go.kr/dg/lib,"매주 월요일 / 국경일, 기타 원장이 필요하다고 인정되는 경우",08:00~22:00,132735
7,131059,가온누리작은도서관,"울산광역시 북구 신천소공원로 15, 108동 1층",052-292-2774,-,35.6454614,129.346506,https://usbl.bukgu.ulsan.kr/,"매주 토요일, 일요일 / 법정공휴일",-,10132
8,111108,가온도서관,서울특별시 중구 동호로 14길 18,02-2230-2921,02-2285-4607,37.5563885,127.0108687,https://www.e-junggulib.or.kr/,매주 월요일 / 법정공휴일,-,78109
9,127065,가창면 참꽃 작은도서관,대구광역시 달성군 가창면 가창로220길 8,053-760-7731,-,35.8023213,128.623414,http://dalseong.daegu.kr/culture/index.do?menu...,-,-,18438


In [248]:
seoul_lib_list = []
for i in range(0,len(lib_df)):
    seoul_lib_list.append(lib_df.iloc[i]['address'].split()[0])

In [249]:
lib_df['city'] = seoul_lib_list

In [253]:
seoul_lib_df = lib_df[lib_df['city'] == '서울특별시']

In [267]:
seoul_lib_df['libCode'].iloc[0]

'111456'

In [270]:
for i in seoul_lib_df['libCode']:
    lib_rental_code_url = 'http://data4library.kr/api/usageTrend?authKey=' +auth_key+'&libCode='+str(i)+'&type=D'
    break



In [272]:
lib_rental_code_url

'http://data4library.kr/api/usageTrend?authKey=7d39c6d3dcf678df9fbe84c27a7d978e0e534a12568b2d8be32af2a2cd38e5b5&libCode=111456&type=D'

In [ ]:
http://data4library.kr/api/itemSrch?type=ALL&libCode=111456&authKey=7d39c6d3dcf678df9fbe84c27a7d978e0e534a12568b2d8be32af2a2cd38e5b5&pageNo=1&page
Size=10

In [ ]:
http://data4library.kr/api/itemSrch?type=ALL&libCode=111456&authKey=7d39c6d3dcf678df9fbe84c27a7d978e0e534a12568b2d8be32af2a2cd38e5b5&pageNo=1&page
Size=10

# 초중고

In [274]:
school_df= pd.read_csv('서울시_초중고_위치데이터.csv')

In [277]:
# 도서관 위/경도를 통해 해당하는 동 추출
dong_name = []
number = []
for i in range(0,len(school_df)):
    for j in range(0,len(seoul_geo_df)):
        test_code1 = Point(school_df.iloc[i]['경도'], school_df.iloc[i]['위도'])
        if test_code1.within(seoul_geo_df.iloc[j]['geometry']):
            dong_name.append(seoul_geo_df.iloc[j]['adm_nm'])
#             print(i)
            number.append(i)
#             print(seoul_geo_df.iloc[j]['adm_nm'])
            break


# 동 이름만 추출 + 도서관 데이터에 동 이름 추가 
dong=[]
for i in dong_name:
    dong.append(i.split()[-1])
school_df['행정동'] = dong
